In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from opencensus.metrics.export.gauge import LongGauge
from opencensus.metrics.label_key import LabelKey


lg = LongGauge("name", "description", "unit",
               [LabelKey("lk1", "first label key"), LabelKey("lk2", "second label key")])
lg

LongGauge(descriptor.name="name", points=OrderedDict())

In [3]:
# no points, null metric
from datetime import datetime as dt

lg.get_metric(dt.now()) is None

True

In [4]:
# we get a different point for each set of label values
lp_en = lg.get_time_series(["one", "two"])
lp_es = lg.get_time_series(["uno", "dos"])

lp_en, lp_es

(GaugePointLong(0), GaugePointLong(0))

In [5]:
# check that we created default (0-value) points and can now get a metric
lg.get_metric(dt.now())

Metric((lk1="one", lk2="two", [0]), (lk1="uno", lk2="dos", [0]), descriptor.name="name")

In [6]:
# points are cached on the gauge, one per set of label values
assert lp_en == lg.get_time_series(["one", "two"])

In [7]:
# check that we can't add a double val to a long gauge
lp_en.add(10.0)

ValueError: GaugePointLong only supports integer types

In [8]:
lp_en.add(10)
lp_en, lp_es

(GaugePointLong(10), GaugePointLong(0))

In [9]:
lp_en.add(10)
lp_es.set(1)
lp_es.set(2)
lp_es.set(-1)
lp_en, lp_es

(GaugePointLong(20), GaugePointLong(-1))

In [10]:
met = lg.get_metric(dt.now())
met

Metric((lk1="one", lk2="two", [20]), (lk1="uno", lk2="dos", [-1]), descriptor.name="name")

In [11]:
met.time_series[0]

TimeSeries([20], label_values=('one', 'two'), start_timestamp=2019-02-06 20:00:11.529178)

In [12]:
met.time_series[0].points[0].value

ValueLong(20)

In [13]:
from opencensus.metrics.export.gauge import DoubleGauge

dg = DoubleGauge("name", "description", "unit",
                 [LabelKey("lk1", "first label key"), LabelKey("lk2", "second label key")])
dg

DoubleGauge(descriptor.name="name", points=OrderedDict())

In [14]:
dp1 = dg.get_time_series(["yi", "er"])
dp1.add(2.5)
dp1

GaugePointDouble(2.5)

In [15]:
# check that we get the same point
dp2 = dg.get_time_series(["yi", "er"])
dp2.add(2.5)
assert dp1 == dp2
dp2

GaugePointDouble(5.0)